In [ ]:
%matplotlib notebook
#%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

In [ ]:
# libraries
import sys
import xml.etree.ElementTree as et
from xml.etree.ElementTree import XMLParser
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.colors as colors
import matplotlib.cm as cmx
import pandas as pd
from IPython import display

In [ ]:
l1_memory_blocks_sharc = ['L1 Block 0 SRAM', 'L1 Block 1 SRAM', 'L1 Block 2 SRAM', 'L1 Block 3 SRAM']
memory_blocks_21584 = ['L1 Block 0 SRAM', 'L1 Block 1 SRAM', 'L1 Block 2 SRAM', 'L1 Block 3 SRAM', 'L2 RAM']
memory_blocks_21569 = ['L1 Block 0 SRAM', 'L1 Block 1 SRAM', 'L1 Block 2 SRAM', 'L1 Block 3 SRAM', 'L2 RAM', 'DDR RAM']

In [ ]:
def my_ceil_div(x, y):
    return int(math.ceil(float(x)/y))

class ldf_input_section:
    def __init__(self, name, start, size, width, input_file):
        self.name = name
        self.start = int(start, 0)
        self.width = width
        self.size = int(size, 0)
        self.end = self.start + self.size - 1;
        self.size_bytes = self.size * (self.width/8)
        self.input_file = input_file
        
    def myprint(self):
        print 'start: %s 0x%8.8X size 0x%X file %s' % (self.name, self.start, self.size, self.input_file)
    
class ldf_output_section:
    def __init__(self, start, size, width, debug=False):
        self.start = int(start, 0)
        self.width = width
        self.size = int(size, 0)
        self.end = self.start + self.size - 1;
        self.size_bytes = self.size * (self.width/8)
        self.finalized = False
        self.INPUT_SECTIONS = {}
        self.debug = debug

    def set_address_ranges(self, BW, SW, NW, ISA, LW):
        self.BWstart  = BW
        self.SWstart  = SW
        self.NWstart  = NW
        self.ISAstart = ISA
        self.LWstart  = LW
        
        self.BWend  = BW  + my_ceil_div(self.size_bytes , (8 /8)) - 1
        self.SWend  = SW  + my_ceil_div(self.size_bytes , (16/8)) - 1
        self.NWend  = NW  + my_ceil_div(self.size_bytes , (32/8)) - 1
        self.ISAend = ISA + my_ceil_div(self.size_bytes , (48/8)) - 1
        self.LWend  = LW  + my_ceil_div(self.size_bytes , (64/8)) - 1
        
    def add_input_section(self, name, start, size, input_file):
        self.INPUT_SECTIONS[start] = ldf_input_section(name, start, size, self.width, input_file)
        return self.INPUT_SECTIONS[start]
        
    def scan_input_sections_tree(self, root):
        self.root = root
        for ins in self.root.iter('INPUT_SECTION'):
            name = ins.get('name')            
            size = ins.get('size')
            if int(size, 0) == 0:
                continue
            start = ins.get('start_address')
            input_file = ins.find('INPUT_FILE').text
            my_input_section = self.add_input_section(name, start, size, input_file)
            if self.debug:
                print '\t\t\t' + name + '\t' + input_file
        
    def set_finalized(self):
        self.addresses = sorted(self.INPUT_SECTIONS.keys(), cmp=lambda x, y: cmp(self.INPUT_SECTIONS[x].start, self.INPUT_SECTIONS[y].start))
        self.count = len(self.INPUT_SECTIONS)
        self.finalized = True

    def get_input_sections_count(self):
        if self.finalized:
            return self.count
        else:
            return None            

    def get_input_section_at_address(self, address):
        if self.finalized:
            if address in self.addresses:
                return self.INPUT_SECTIONS[address]
        else:
            return None
    
    def get_input_section_at_index(self, idx):
        if self.finalized:
            if idx < self.count:
                return self.INPUT_SECTIONS[self.addresses[idx]]
        else:
            return None
        
    def get_input_sections(self):
        if self.finalized:
            return self.addresses
        else:
            return None      
            
    def myprint(self):
        if self.finalized == False:
            return        
        
        print '\t\tstart 0x%8.8X size 0x%X width 0x%X' % (self.start, self.size, self.width)
        for address in self.addresses:
            print '\t\t\t',
            self.INPUT_SECTIONS[address].myprint()
        
class ldf_memory_section:
    def __init__(self, start, end, width, usage, debug=False):
        self.start = int(start, 0)
        self.end = int(end, 0)
        self.width = width
        self.usage = int(usage, 0)
        self.size = (self.end - self.start + 1)
        self.size_bytes = self.size * (self.width/8)
        self.usage_bytes = self.usage * (self.width/8)
        self.finalized = False
        self.debug = debug
        
        self.OUTPUT_SECTIONS = {}

    def set_address_ranges(self, BW, SW, NW, ISA, LW):
        self.BWstart  = BW
        self.SWstart  = SW
        self.NWstart  = NW
        self.ISAstart = ISA
        self.LWstart  = LW
        
        self.BWend  = BW  + my_ceil_div(self.size_bytes , (8 /8)) - 1
        self.SWend  = SW  + my_ceil_div(self.size_bytes , (16/8)) - 1
        self.NWend  = NW  + my_ceil_div(self.size_bytes , (32/8)) - 1
        self.ISAend = ISA + my_ceil_div(self.size_bytes , (48/8)) - 1
        self.LWend  = LW  + my_ceil_div(self.size_bytes , (64/8)) - 1
        
    def add_output_section(self, name, start, size, width):
        self.OUTPUT_SECTIONS[name] = ldf_output_section(start, size, width, debug=self.debug)
        
        start = int(start,0)
        offset_bytes = None
        if (width == 8):
            offset_bytes = (start - self.BWstart) * (width/8)
        elif (width == 16):
            offset_bytes = (start - self.SWstart) * (width/8)
        elif (width == 32):
            offset_bytes = (start - self.NWstart) * (width/8)
        elif (width == 48):
            offset_bytes = (start - self.ISAstart) * (width/8)
        elif (width == 64):
            offset_bytes = (start - self.LWstart) * (width/8)
        if offset_bytes != None:
            BW  = self.BWstart  + my_ceil_div(offset_bytes , (8/8))
            SW  = self.SWstart  + my_ceil_div(offset_bytes , (16/8))
            NW  = self.NWstart  + my_ceil_div(offset_bytes , (32/8))
            ISA = self.ISAstart + my_ceil_div(offset_bytes , (48/8))
            LW  = self.LWstart  + my_ceil_div(offset_bytes , (64/8))
        
        self.OUTPUT_SECTIONS[name].set_address_ranges(BW, SW, NW, ISA, LW)
        
        return self.OUTPUT_SECTIONS[name]
        
    def isMember(self, _start, _end, width):
        start = int(_start,0)
        end = int(_end,0)        
        
        if (width == 8):
            s0 = self.BWstart
            e0 = self.BWend
        elif (width == 16):
            s0 = self.SWstart
            e0 = self.SWend
        elif (width == 32):
            s0 = self.NWstart
            e0 = self.NWend
        elif (width == 48):
            s0 = self.ISAstart
            e0 = self.ISAend
        elif (width == 64):
            s0 = self.LWstart
            e0 = self.LWend
        if (start >= s0) and (end <= e0):
            return True
        else:
            return False

    def findMemoryWidth(self, _addr):
        addr = int(_addr,0)
        if (addr >= self.BWstart) and (addr <= self.BWend):
            return 8
        elif (addr >= self.SWstart) and (addr <= self.SWend):
            return 16
        elif (addr >= self.NWstart) and (addr <= self.NWend):
            return 32
        elif (addr >= self.ISAstart) and (addr <= self.ISAend):
            return 48
        elif (addr >= self.LWstart) and (addr <= self.LWend):
            return 64
        else:
            return None
        
    def scan_output_sections_tree(self, root):
        self.root = root
        for os in self.root.iter('OUTPUT_SECTION'):
            name = os.get('name')            
            if self.debug:
                print '\t\t' + name
            size = os.get('word_size')
            if int(size, 0) == 0:
                continue
            start = os.get('start_address')
            if os.get('memory_width') == None:
                width = self.findMemoryWidth(start)
            else:
                width = int(os.get('memory_width'),0)
            my_output_section = self.add_output_section(name, start, size, width)
            my_output_section.scan_input_sections_tree(os) 
            my_output_section.set_finalized()
            
    def add_heap_block(self, root):
        for ldf_symbol in root.iter('LDF_SYMBOLS'):
            start = None
            end = None                
            for symbol in ldf_symbol.iter('SYMBOL'):
                if symbol.get('name') == 'ldf_heap_space':
                    start = symbol.get('address')
                if symbol.get('name') == 'ldf_heap_end':
                    end = symbol.get('address')
                if symbol.get('name') == 'ldf_heap_length':
                    length = symbol.get('address')
            if (start != None) and (end != None):
                if self.isMember(start, end, 8):
                    if self.debug:
                        print '\t' + 'Heap'
                    my_output_section = self.add_output_section('Heap', start, length, 8)
                    my_output_section.set_finalized()
            
    def add_stack_block(self, root):
        for ldf_symbol in root.iter('LDF_SYMBOLS'):
            start = None
            end = None
            for symbol in ldf_symbol.iter('SYMBOL'):
                if symbol.get('name') == 'ldf_stack_space':
                    start = symbol.get('address')
                if symbol.get('name') == 'ldf_stack_end':
                    end = symbol.get('address')
                if symbol.get('name') == 'ldf_stack_length':
                    length = symbol.get('address')
            if (start != None) and (end != None):
                if self.isMember(start, end, 8):
                    if self.debug:
                        print '\t' + 'Stack'
                    my_output_section = self.add_output_section('Stack', start, length, 8)                        
                    my_output_section.set_finalized()
                
    def set_finalized(self):
        self.names = sorted(self.OUTPUT_SECTIONS.keys(), cmp=lambda x, y: cmp(self.OUTPUT_SECTIONS[x].BWstart, self.OUTPUT_SECTIONS[y].BWstart))
        self.count = len(self.OUTPUT_SECTIONS)
        self.finalized = True

    def get_output_sections_count(self):
        if self.finalized:
            return self.count
        else:
            return None            

    def get_output_section_by_name(self, name):
        if self.finalized:
            if name in self.names:
                return self.OUTPUT_SECTIONS[name]
        else:
            return None
    
    def get_output_section_at_index(self, idx):
        if self.finalized:
            if idx < self.count:
                return self.OUTPUT_SECTIONS[self.names[idx]]
        else:
            return None
        
    def get_output_sections(self):
        if self.finalized:
            return self.names
        else:
            return None      
            
    def myprint(self):
        if self.finalized == False:
            return        
        
        print '\tstart 0x%8.8X end 0x%8.8X usage 0x%X width 0x%X' % (self.start, self.end, self.usage, self.width)
        for name in self.names:
            print '\t\t' + name
            self.OUTPUT_SECTIONS[name].myprint()
            print ''
            
class ldf_memory_range:
    def __init__(self, start, end, width):
        self.width = width
        self.start = int(start,0)
        self.end = int(end,0)
        self.size = (self.end - self.start + 1)
        self.size_bytes = self.size * (self.width/8)
        
class ldf_memory_block:
    def __init__(self, debug=False):
        self.LW = None
        self.ISA = None
        self.NW = None
        self.SW = None
        self.BW = None
        self.cache = None
        self.MEMORY_SECTIONS = {}
        self.finalized = False
        self.debug = debug
        
    def set_cache_type(self, cache):
        self.cache = cache
        if cache != None:
            if cache == 'DM':
                self.cache_symbol = '___ldf_dmcachesize'
            elif cache == 'PM':
                self.cache_symbol = '___ldf_pmcachesize'
            elif cache == 'I':
                self.cache_symbol = '___ldf_icachesize'
        
    def long_word_address(self,start,end):
        self.LW = ldf_memory_range(start, end, 64)
        
    def ISA_code_address(self,start,end):
        self.ISA = ldf_memory_range(start, end, 48)
        
    def normal_word_address(self,start,end):
        self.NW = ldf_memory_range(start, end, 32)
        
    def short_word_address(self,start,end):
        self.SW = ldf_memory_range(start, end, 16)
        
    def byte_access_address(self,start,end):
        self.BW = ldf_memory_range(start, end, 8)
        
    def isMember(self, _start, _end, width):
        start = int(_start,0)
        end = int(_end,0)        
        
        if (width == 8):
            s0 = self.BW.start
            e0 = self.BW.end
        elif (width == 16):
            s0 = self.SW.start
            e0 = self.SW.end
        elif (width == 32):
            s0 = self.NW.start
            e0 = self.NW.end
        elif (width == 48):
            s0 = self.ISA.start
            e0 = self.ISA.end
        elif (width == 64):
            s0 = self.LW.start
            e0 = self.LW.end
        if (start >= s0) and (end <= e0):
            return True
        else:
            return False
        
    def add_memory_section(self, name, start, end, width, usage):
        self.MEMORY_SECTIONS[name] = ldf_memory_section(start, end, width, usage, debug=self.debug)
        
        start = int(start,0)
        offset_bytes = None
        
        if (width == 8):
            offset_bytes = (start - self.BW.start) * (width/8)
        elif (width == 16):
            offset_bytes = (start - self.SW.start) * (width/8)
        elif (width == 32):
            offset_bytes = (start - self.NW.start) * (width/8)
        elif (width == 48):
            offset_bytes = (start - self.ISA.start) * (width/8)
        elif (width == 64):
            offset_bytes = (start - self.LW.start) * (width/8)
        if offset_bytes != None:
            BW  = self.BW.start  + my_ceil_div(offset_bytes , (8/8))
            SW  = self.SW.start  + my_ceil_div(offset_bytes , (16/8))
            NW  = self.NW.start  + my_ceil_div(offset_bytes , (32/8))
            ISA = self.ISA.start + my_ceil_div(offset_bytes , (48/8))
            LW  = self.LW.start  + my_ceil_div(offset_bytes , (64/8))
        
        self.MEMORY_SECTIONS[name].set_address_ranges(BW, SW, NW, ISA, LW)
        return self.MEMORY_SECTIONS[name]
        
    def scan_memory_sections_tree(self, root):
        self.root = root
        for ms in self.root.iter('MEMORY'):
            name = ms.get('name')
            start = ms.get('start_address')
            end = ms.get('end_address')
            width = int(ms.get('width'),0)
            usage = ms.get('words_used')
            if self.isMember(start, end, width):
                if self.debug:
                    print '\t' + name
                my_memory_section = self.add_memory_section(name, start, end, width, usage)
                my_memory_section.scan_output_sections_tree(ms)
                my_memory_section.add_heap_block(self.root)
                my_memory_section.add_stack_block(self.root)
                my_memory_section.set_finalized()
                
    def add_cache_block(self, root):
        for ldf_symbol in root.iter('LDF_SYMBOLS'):
            for symbol in ldf_symbol.iter('SYMBOL'):
                if self.cache == None:
                    continue
                if symbol.get('name') == self.cache_symbol:
                    address = symbol.get('address')
                    size = None
                    if address == '0xffffffff':
                        size = 0 * 1024
                    elif address == '0x0':
                        size = 16 * 1024
                    elif address == '0x1':
                        size = 32 * 1024
                    elif address == '0x2':
                        size = 64 * 1024
                    elif address == '0x3':
                        size = 128 * 1024
                    if size != None:
                        end = '0x%8.8X' % self.BW.end
                        start = '0x%8.8X' % (self.BW.end - size + 1)
                        width = 8
                        usage = '0x0'
                        name = self.cache + 'cache'
                        my_memory_section = self.add_memory_section(name, start, end, width, usage)
                        my_memory_section.set_finalized()
            
    def set_finalized(self):
        self.names = sorted(self.MEMORY_SECTIONS.keys(), cmp=lambda x, y: cmp(self.MEMORY_SECTIONS[x].BWstart, self.MEMORY_SECTIONS[y].BWstart))        
        self.count = len(self.MEMORY_SECTIONS)
        self.finalized = True

    def get_memory_sections_count(self):
        if self.finalized:
            return self.count
        else:
            return None            

    def get_memory_section_by_name(self, name):
        if self.finalized:
            if name in self.names:
                return self.MEMORY_SECTIONS[name]
        else:
            return None
    
    def get_memory_section_at_index(self, idx):
        if self.finalized:
            if idx < self.count:
                return self.MEMORY_SECTIONS[self.names[idx]]
        else:
            return None
    
    def get_memory_sections(self):
        if self.finalized:
            return self.names
        else:
            return None      
            
    def myprint(self):
        if self.finalized == False:
            return
        
        print 'start 0x%8.8X end 0x%8.8X size 0x%X width 0x%X' % (self.BW.start, self.BW.end, self.BW.size, self.BW.width)
        for name in self.names:
            print '\t' + name
            self.MEMORY_SECTIONS[name].myprint()
            print ''

class ldf_memory_blocks:
    def __init__(self, proc, xml_root, debug=False):
        self.debug = debug
        self.proc = proc
        self.MEMORY_BLOCKS = {}
        
        mb = ldf_memory_block(debug=self.debug)
        mb.long_word_address  ('0x00048000','0x0004DFFF')
        mb.ISA_code_address   ('0x00090000','0x00097FFF')
        mb.normal_word_address('0x00090000','0x0009BFFF')
        mb.short_word_address ('0x00120000','0x00137FFF')
        mb.byte_access_address('0x00240000','0x0026FFFF')
        mb.set_cache_type(None)
        self.MEMORY_BLOCKS['L1 Block 0 SRAM'] = mb
        
        mb = ldf_memory_block(debug=self.debug)
        mb.long_word_address  ('0x00058000','0x0005DFFF')
        mb.ISA_code_address   ('0x000B0000','0x000B7FFF')
        mb.normal_word_address('0x000B0000','0x000BBFFF')
        mb.short_word_address ('0x00160000','0x00177FFF')
        mb.byte_access_address('0x002C0000','0x002EFFFF')
        mb.set_cache_type('DM')
        self.MEMORY_BLOCKS['L1 Block 1 SRAM'] = mb
        
        mb = ldf_memory_block(debug=self.debug)
        mb.long_word_address  ('0x00060000','0x00063FFF')
        mb.ISA_code_address   ('0x000C0000','0x000C5554')
        mb.normal_word_address('0x000C0000','0x000C7FFF')
        mb.short_word_address ('0x00180000','0x0018FFFF')
        mb.byte_access_address('0x00300000','0x0031FFFF')
        mb.set_cache_type('PM')
        self.MEMORY_BLOCKS['L1 Block 2 SRAM'] = mb
        
        mb = ldf_memory_block(debug=self.debug)
        mb.long_word_address  ('0x00070000','0x00073FFF')
        mb.ISA_code_address   ('0x000E0000','0x000E5554')
        mb.normal_word_address('0x000E0000','0x000E7FFF')
        mb.short_word_address ('0x001C0000','0x001CFFFF')
        mb.byte_access_address('0x00380000','0x0039FFFF')
        mb.set_cache_type('I')
        self.MEMORY_BLOCKS['L1 Block 3 SRAM'] = mb
        
        if proc == 'ADSP-21584':
            mb = ldf_memory_block(debug=self.debug)
            mb.long_word_address  ('0x00000000','0x00000000')
            mb.ISA_code_address   ('0x005A0000','0x005AAAAF')
            mb.normal_word_address('0x08020000','0x0802FFFF')
            mb.short_word_address ('0x00BA0000','0x00BBFFFF')
            mb.byte_access_address('0x20080000','0x200BFFFF')
            self.MEMORY_BLOCKS['L2 RAM'] = mb 
        
        if proc == 'ADSP-21569':
            mb = ldf_memory_block(debug=self.debug)
            mb.long_word_address  ('0x00000000','0x00000000')
            mb.ISA_code_address   ('0x005C0000','0x005EAAAA')
            mb.normal_word_address('0x08000000','0x0803FFFF')
            mb.short_word_address ('0x00B80000','0x00BFFFFF')
            mb.byte_access_address('0x20000000','0x200FFFFF')
            self.MEMORY_BLOCKS['L2 RAM'] = mb
            
            mb = ldf_memory_block(debug=self.debug)
            mb.long_word_address  ('0x00000000','0x00000000')
            mb.ISA_code_address   ('0x00000000','0x00000000')
            mb.normal_word_address('0x00000000','0x00000000')
            mb.short_word_address ('0x00000000','0x00000000')
            mb.byte_access_address('0x80000000','0xbfffffff')
            self.MEMORY_BLOCKS['DDR RAM'] = mb        
        
        self.names = sorted(self.MEMORY_BLOCKS.keys(), cmp=lambda x, y: cmp(self.MEMORY_BLOCKS[x].BW.start, self.MEMORY_BLOCKS[y].BW.start))
        
        self.count = len(self.MEMORY_BLOCKS)
        
        self.scan_map_xml(xml_root)        

    def get_memory_blocks_count(self):
        return self.count

    def get_memory_block_by_name(self, name):
        if name in self.names:
            return self.MEMORY_BLOCKS[name]
        else:
            return None
    
    def get_memory_block_at_index(self, idx):
        if idx < self.count:
            return self.MEMORY_BLOCKS[self.names[idx]]
        else:
            return None
    
    def get_memory_blocks(self):
        return self.names

    def myprint(self):
        for name in self.names:
            print name
            self.MEMORY_BLOCKS[name].myprint()
            
    def scan_map_xml(self, xml_root):
        self.root = xml_root
        for mb in self.get_memory_blocks():
            if self.debug:
                print mb
            self.MEMORY_BLOCKS[mb].scan_memory_sections_tree(self.root)
            self.MEMORY_BLOCKS[mb].add_cache_block(self.root)
            self.MEMORY_BLOCKS[mb].set_finalized()

In [ ]:
def img_l1_memory_blocks_usage(img, l1_memory_blocks):        
    mb = [None, None, None, None]
    ms_cnt = [0, 0, 0, 0]
    ms_names = ['', '', '', '']
    mscnt_max = 0
    for n in range(4):
        #print n
        mb[n] = img.get_memory_block_by_name(l1_memory_blocks[n])
        ms_names[n] = mb[n].get_memory_sections()
        ms_cnt[n] = mb[n].get_memory_sections_count()
        mscnt_max = max(mscnt_max, ms_cnt[n])
        #print ms_cnt[n]
        #print ms_names[n]

    #print mscnt_max

    # https://matplotlib.org/2.0.0/examples/color/named_colors.html
    unused_color = 'skyblue'
    used_colors = ['tan', 'mediumpurple', 'sandybrown', 'darkcyan', 'slategray', 'hotpink', 'indigo']
    cache_color = 'orangered'
    barWidth = 0.6
    # The position of the bars on the x-axis
    r = [0,1,2,3]
    
    if mscnt_max > len(used_colors):
        print 'Error: add some colors to used_colors'
        return

    fig = plt.figure(num=None, figsize=(8, 8), dpi=100, facecolor='w', edgecolor='k')
    axs = [None, None]
    axs[1] = fig.add_axes([0.1,0.1,0.8,0.2])
    axs[0] = fig.add_axes([0.1,0.4,0.8,0.5])

    bar_bottom = [0, 0, 0, 0]
    cell_text = []
    cell_color = []
    rows = []
    for idx in range(mscnt_max):
        #print idx
        bar_used  = [0, 0, 0, 0]
        bar_unused  = [0, 0, 0, 0]
        row_used = ['', '', '', '']
        row_unused = ['', '', '', '']
        for n in range(4):
            if (idx < ms_cnt[n]) and ('cache' not in ms_names[n][idx]):
                ms  = mb[n].get_memory_section_at_index(idx)
                #print "%s size %d used %d" % (ms_names[n][idx], ms.size_bytes, ms.usage_bytes)
                bar_used[n] = ms.usage_bytes
                bar_unused[n] = ms.size_bytes - ms.usage_bytes
                if bar_used[n] > 0:
                    row_used[n] = "%s %.2fKB" % (ms_names[n][idx], bar_used[n]/1024.0)
                if bar_unused[n] > 0:
                    row_unused[n] = "%s %.2fKB" % (ms_names[n][idx], bar_unused[n]/1024.0)
        if sum(bar_used) > 0:
            axs[0].bar(r, bar_used, bottom=bar_bottom, color=used_colors[idx], edgecolor='white', width=barWidth)
            bar_bottom = np.add(bar_bottom, bar_used).tolist()
            cell_text.append(row_used)
            rows.append('used')

            row_color = ['white','white','white','white']
            for n in range(4):
                if bar_used[n] != 0:
                    row_color[n] = used_colors[idx]
            cell_color.append(row_color)

        if sum(bar_unused) > 0:
            axs[0].bar(r, bar_unused, bottom=bar_bottom, color=unused_color, edgecolor='white', width=barWidth)
            bar_bottom = np.add(bar_bottom, bar_unused).tolist()
            cell_text.append(row_unused)
            rows.append('unused')

            row_color = ['white','white','white','white']
            for n in range(4):
                if bar_unused[n] != 0:
                    row_color[n] = unused_color
            cell_color.append(row_color)

    bar_unused  = [0, 0, 0, 0]
    row_unused = ['', '', '', '']
    cache_names = ['','DMcache', 'PMcache', 'Icache']
    for n in range(1,4):
        ms  = mb[n].get_memory_section_by_name(cache_names[n])
        #print "%s size %d used %d" % (cache_names[n], ms.size_bytes, ms.usage_bytes)
        if ms != None:
            bar_unused[n] = ms.size_bytes
        if bar_unused[n] > 0:
            row_unused[n] = "%s %.2fKB" % (cache_names[n], bar_unused[n]/1024.0)
    
    if sum(bar_unused) > 0:
        axs[0].bar(r, bar_unused, bottom=bar_bottom, color=cache_color, edgecolor='white', width=barWidth)
        cell_text.append(row_unused)
        rows.append('cache')
        bar_bottom = np.add(bar_bottom, bar_unused).tolist()
    
        row_color = ['white','white','white','white']
        for n in range(4):
            if bar_unused[n] != 0:
                row_color[n] = cache_color
        cell_color.append(row_color)
    
    bar_max = max(bar_bottom)
    axs[0].set_yticks(np.arange(0, bar_max, step=32*1024))
    axs[0].set_xticks(r)
    axs[0].set_xticklabels(l1_memory_blocks)
    axs[0].tick_params('x', labelrotation = 10)
    axs[0].grid(axis='y')
    
    rows = rows[::-1]
    cell_text.reverse()
    cell_color.reverse()
    #print cell_text
    #print cell_color

    axs[1].table(cellText=cell_text, rowLabels=rows, cellColours=cell_color,colLabels=l1_memory_blocks,loc='center')
    axs[1].axis('tight')
    axs[1].axis('off')
    # Adjust layout to make room for the table:
    #plt.subplots_adjust(right=0.2, top=0.2)
    fig.suptitle('L1 memory blocks', fontsize=16)

    # Show graphic
    plt.show()
    df = pd.DataFrame(cell_text, index=rows, columns=l1_memory_blocks)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 256):
        display.display(df)      
    

In [ ]:
def img_memory_blocks_usage(img, memory_block):
    mb = img.get_memory_block_by_name(memory_block)
    if 'DDR' in memory_block:
        size_div = (1024.0*1024.0)
        size_str = 'MB'
    else:
        size_div = (1024.0)
        size_str = 'KB'
    
    ms_names = mb.get_memory_sections()
    ms_cnt = mb.get_memory_sections_count()
    
    ms = [None for msidx in range(ms_cnt)]
    os_cnt = [0 for msidx in range(ms_cnt)]
    os_names = [None for msidx in range(ms_cnt)]
    oscnt_max = 0
    
    for n in range(ms_cnt):
        ms[n] = mb.get_memory_section_at_index(n)
        os_names[n] = ms[n].get_output_sections()
        os_cnt[n] = ms[n].get_output_sections_count()
        oscnt_max = max(oscnt_max, os_cnt[n])
    
    # https://matplotlib.org/2.0.0/examples/color/named_colors.html
    unused_color = 'skyblue'
    used_colors = ['tan', 'mediumpurple', 'sandybrown', 'darkcyan', 'slategray', 'hotpink', 'indigo']
    cache_color = 'orangered'
    barWidth = 0.6
    
    # The position of the bars on the x-axis
    r = range(ms_cnt)
    
    if oscnt_max > len(used_colors):
        print 'Error: add some colors to used_colors'
        return

    fig = plt.figure(num=None, figsize=(8, 8), dpi=100, facecolor='w', edgecolor='k')
    axs = [None, None]
    axs[1] = fig.add_axes([0.1,0.1,0.8,0.2])
    axs[0] = fig.add_axes([0.1,0.4,0.8,0.5])

    bar_bottom = [0 for msidx in range(ms_cnt)]
    cell_text = []
    cell_color = []
    rows = []
    for idx in range(oscnt_max):
        bar_used  = [0 for msidx in range(ms_cnt)]
        row_used = ['' for msidx in range(ms_cnt)]
        for n in range(ms_cnt):
            if idx < os_cnt[n]:
                os  = ms[n].get_output_section_at_index(idx)
                bar_used[n] = os.size_bytes/size_div
                if bar_used[n] > 0:
                    row_used[n] = "%s %.2f%s" % (os_names[n][idx], bar_used[n], size_str)
                    
        if sum(bar_used) > 0:
            axs[0].bar(r, bar_used, bottom=bar_bottom, color=used_colors[idx], edgecolor='white', width=barWidth)
            bar_bottom = np.add(bar_bottom, bar_used).tolist()
            cell_text.append(row_used)
            rows.append('used')

            row_color = ['white' for msidx in range(ms_cnt)]
            for n in range(ms_cnt):
                if bar_used[n] != 0:
                    row_color[n] = used_colors[idx]
            cell_color.append(row_color)

    bar_unused = [0 for msidx in range(ms_cnt)]
    row_unused = ['' for msidx in range(ms_cnt)] 
    for n in range(ms_cnt):
        if 'cache' in ms_names[n]:
            continue
        bar_unused[n] = (ms[n].size_bytes - ms[n].usage_bytes)/size_div
        if bar_unused[n] > 0:
            row_unused[n] = "%s %.2f%s" % (ms_names[n], bar_unused[n], size_str)
    
    if sum(bar_unused) > 0:
        axs[0].bar(r, bar_unused, bottom=bar_bottom, color=unused_color, edgecolor='white', width=barWidth)
        cell_text.append(row_unused)
        rows.append('unused')
        bar_bottom = np.add(bar_bottom, bar_unused).tolist()
    
        row_color = ['white' for msidx in range(ms_cnt)]
        for n in range(ms_cnt):
            if bar_unused[n] != 0:
                row_color[n] = unused_color
        cell_color.append(row_color)
    
    bar_unused = [0 for msidx in range(ms_cnt)]
    row_unused = ['' for msidx in range(ms_cnt)] 
    for n in range(ms_cnt):
        if 'cache' not in ms_names[n]:
            continue
        bar_unused[n] = (ms[n].size_bytes - ms[n].usage_bytes)/size_div
        if bar_unused[n] > 0:
            row_unused[n] = "%s %.2f%s" % (ms_names[n], bar_unused[n], size_str)
    
    if sum(bar_unused) > 0:
        axs[0].bar(r, bar_unused, bottom=bar_bottom, color=cache_color, edgecolor='white', width=barWidth)
        cell_text.append(row_unused)
        rows.append('cache')
        bar_bottom = np.add(bar_bottom, bar_unused).tolist()
    
        row_color = ['white' for msidx in range(ms_cnt)]
        for n in range(ms_cnt):
            if bar_unused[n] != 0:
                row_color[n] = cache_color
        cell_color.append(row_color)
        
    if 'L1' in memory_block:
        mystep = 32
    elif 'L2' in memory_block:
        mystep = 128
    else: #'DDR'
        mystep = 128
        
    axs[0].set_yticks(np.arange(0, (mb.BW.size/size_div), step=mystep))
    axs[0].set_ylabel(size_str)
    axs[0].set_xticks(r)
    axs[0].set_xticklabels(ms_names)
    axs[0].tick_params('x', labelrotation = 10)
    axs[0].grid(axis='y')
    
    rows = rows[::-1]
    cell_text.reverse()
    cell_color.reverse()
    #print cell_text
    #print cell_color

    axs[1].table(cellText=cell_text, rowLabels=rows, cellColours=cell_color,colLabels=ms_names,loc='center')
    axs[1].axis('tight')
    axs[1].axis('off')
    # Adjust layout to make room for the table:
    #plt.subplots_adjust(right=0.2, top=0.2)
    fig.suptitle(memory_block, fontsize=16)

    # Show graphic
    plt.show()
    df = pd.DataFrame(cell_text, index=rows, columns=ms_names)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 256):
        display.display(df)  

In [ ]:
def img_memory_section_usage(img, memory_blocks, memory_section):
    parent = None
    for memory_block in memory_blocks:
        mb = img.get_memory_block_by_name(memory_block)
        if memory_section in mb.get_memory_sections():
            parent = mb
            break
    if parent == None:
        return
    
    mb = parent
    ms = mb.get_memory_section_by_name(memory_section)
    os_names = ms.get_output_sections()
    os_names = [x for x in os_names if 'Heap' not in x]
    os_names = [x for x in os_names if 'Stack' not in x]
    os_cnt = len(os_names)
    if os_cnt == 0:
        print 'Info: No output section present in the memory_section %s' % memory_section
        return
    
    os = [None for osidx in range(os_cnt)]
    isfile_cnt = [0 for osidx in range(os_cnt)]
    isfile_names = [None for osidx in range(os_cnt)]
    isfile_totals = [None for osidx in range(os_cnt)]
    isfilecnt_max = 0
    
    bar_max = 0    
    for n in range(os_cnt):
        os[n] = ms.get_output_section_by_name(os_names[n])
        #print os_names[n]
        
        isfiles = [os[n].get_input_section_at_address(x).input_file.split('[')[0] for x in os[n].get_input_sections()]
        isfiles = set(isfiles)
        
        isfile_cnt[n] = len(isfiles)
        isfile_names[n] = list(isfiles)
        is_total = []
        for isfile in isfiles:
            #print '\t' + isfile    
            is_list = [x for x in range(os[n].get_input_sections_count()) if isfile in os[n].get_input_section_at_index(x).input_file]
            #for idx in is_list:
            #    print '\t\t',
            #    os[n].get_input_section_at_index(idx).myprint()
            is_sizes = [os[n].get_input_section_at_index(x).size_bytes for x in is_list]
            #print '\t 0x%X' % sum(is_sizes)
            is_total.append(sum(is_sizes))
        isfile_totals[n] = is_total
        bar_max = max(bar_max, sum(is_total))
        
    isfilecnt_max = max(isfile_cnt)
    
    size_div = (1024.0)
    size_str = 'KB'
    num_steps = 8
    bar_step = (bar_max/size_div)/num_steps
    if (bar_max/size_div) > 32:
        bar_step = (math.ceil(bar_step/8.0)) * 8.0
        
    barWidth = 0.6
    
    values = range(isfilecnt_max)
    jet = cm = plt.get_cmap('tab20b') 
    cNorm  = colors.Normalize(vmin=0, vmax=values[-1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)    
    
    # The position of the bars on the x-axis
    r = range(os_cnt)

    fig = plt.figure(num=None, figsize=(8, 8), dpi=100, facecolor='w', edgecolor='k')
    axs = [None, None]
    axs[1] = fig.add_axes([0.1,0.1,0.8,0.2])
    axs[0] = fig.add_axes([0.1,0.4,0.8,0.5])

    bar_bottom = [0 for osidx in range(os_cnt)]
    cell_text = []
    cell_color = []
    
    for idx in range(isfilecnt_max):
        bar_used = [0 for osidx in range(os_cnt)]
        row_used = ['' for osidx in range(os_cnt)]
        for n in range(os_cnt):
            if idx < isfile_cnt[n]:
                bar_used[n] = isfile_totals[n][idx]/size_div
                if bar_used[n] > 0:
                    row_used[n] = "%s %.4f%s" % (isfile_names[n][idx], bar_used[n], size_str)
                    
        if sum(bar_used) > 0:
            colorVal = scalarMap.to_rgba(values[idx])
            axs[0].bar(r, bar_used, bottom=bar_bottom, color=colorVal, edgecolor='white', width=barWidth)
            bar_bottom = np.add(bar_bottom, bar_used).tolist()
            cell_text.append(row_used)

            row_color = ['white' for osidx in range(os_cnt)]
            for n in range(os_cnt):
                if bar_used[n] != 0:
                    row_color[n] = colorVal
            cell_color.append(row_color)

    axs[0].set_yticks(np.arange(0, (bar_max/size_div), step=bar_step))
    axs[0].set_ylabel(size_str)
    axs[0].set_xticks(r)
    axs[0].set_xticklabels(os_names)
    axs[0].tick_params('x', labelrotation = 10)
    axs[0].grid(axis='y')
    
    cell_text.reverse()
    cell_color.reverse()
    #print cell_text
    #print cell_color

    axs[1].table(cellText=cell_text, cellColours=cell_color,colLabels=os_names,loc='center')
    axs[1].axis('tight')
    axs[1].axis('off')
    # Adjust layout to make room for the table:
    #plt.subplots_adjust(right=0.2, top=0.2)
    fig.suptitle(memory_section, fontsize=16)

    # Show graphic
    plt.show()
    
    df = pd.DataFrame(cell_text, columns=os_names)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 256):
        display.display(df)  

In [ ]:
def img_memory_sections(img, memory_blocks):
    ms = []
    for memory_block in memory_blocks:
        mb = img.get_memory_block_by_name(memory_block)
        ms.extend(mb.get_memory_sections())
    return [x for x in ms if 'cache' not in x]

In [ ]:
def dsp_memory_usage(xml_path, proc, debug=False):
    if proc == 'ADSP-21584':
        memory_blocks = memory_blocks_21584
        l1_memory_blocks = l1_memory_blocks_sharc
    elif proc == 'ADSP-21569':
        memory_blocks = memory_blocks_21569
        l1_memory_blocks = l1_memory_blocks_sharc
    else:
        return
    
    xml_tree = et.parse(xml_path)
    xml_root = xml_tree.getroot()
    for lp_root in xml_root.iter('LINKPROJECT'):
        img = ldf_memory_blocks(proc, lp_root, debug=debug)
        img_l1_memory_blocks_usage(img, l1_memory_blocks)
        for mb in memory_blocks:
            img_memory_blocks_usage(img, mb)
        for ms in img_memory_sections(img, memory_blocks):
            img_memory_section_usage(img, memory_blocks, ms)

In [ ]:
dsp_memory_usage('c:\\work\\xmlparse\\SHARC3.map.xml','ADSP-21584')

In [ ]:
dsp_memory_usage('C:\\work\\ASD_OpenJADE\\Application\\Basicaudio12-Sim\\branches\\asdoj-93\\build\\debug_wb\\codedataL2_IDP_00_00_00.map.xml','ADSP-21569')
